<a href="https://colab.research.google.com/github/yellowleather/LLM-Fundamentals/blob/main/CrewAI_agent_demo_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CrewAI demo notebook: Research a topic, generate content and get social media ready post and a blog article!

# 📦 Install/Upgrade Dependencies
This cell installs the libraries used in the workflow:
- `openai` (new SDK), `transformers` for model helpers
- `crewai`, `langchain`, `langchain-openai` for agent orchestration
- `crewai-tools` for optional tool helpers
- `requests` for webhooks / HTTP calls

> Tip: After upgrading key packages in Colab, use **Runtime → Restart session** to ensure fresh imports.


In [ ]:
!pip install -qU crewai crewai-tools langchain langchain-core langchain-openai transformers requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 🔐 Core Imports & OpenAI API Key
- Imports common libs and CrewAI/LangChain wrappers.
- Reads your OpenAI key (e.g., from Colab `userdata` or a variable you set).
- Exposes `OPENAI_API_KEY` to the environment.
- Creates a `ChatOpenAI` client for quick single-shot prompts if needed.

> Ensure `openai_api_key` is defined (e.g., `from google.colab import userdata; openai_api_key = userdata.get('OPENAI_API_KEY')`)


In [ ]:
# pip install crewai openai requests
import os
import numpy as np
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI # Updated import
import base64, os, requests
from google.colab import userdata # Import userdata
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool
# from diffusers import StableDiffusionPipeline
import torch
from typing import Annotated # Import Annotated

In [ ]:
import getpass

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [ ]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

In [ ]:
# Use secrets manager to securely store and access the API key
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# llm = ChatOpenAI(model="gpt-4o", openai_api_key=OPENAI_API_KEY)  # Pass API key here
llm = ChatOpenAI(model="gpt-4o", openai_api_key=os.environ["OPENAI_API_KEY"])  # Pass API key here

# 🧩 Helper Functions (Social Copy + Webhook)
- `_make_social_copy_impl(title, summary, platform)` calls an LLM to craft a short post tailored to a platform (LinkedIn/Twitter).
- `_post_via_webhook_impl(text, image_path)` sends text (and optional image) to an external webhook URL from `SOCIAL_WEBHOOK_URL`.

> ⚠️ Set `SOCIAL_WEBHOOK_URL` in your environment if you plan to publish.


# 🧱 Tool Base Class Import (Version-safe)
- Tries `crewai.tools.BaseTool`, then `crewai_tools.BaseTool`, then a minimal fallback class.
- This keeps the notebook working across CrewAI versions.


# 🛠️ Class-Based Tools for CrewAI
- `MakeSocialCopyTool`: wraps `_make_social_copy_impl`.
- `PostViaWebhookTool`: wraps `_post_via_webhook_impl`.

These tools can be attached to agents so tasks can call them.


In [ ]:
import os, requests
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [ ]:
from crewai.tools import BaseTool

In [ ]:
# ---------- Concrete implementations ----------

def _make_social_copy_impl(title: str, summary: str, platform: str = "linkedin") -> str:
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = (
        f"Craft a concise {platform} post promoting an article.\n"
        f"Title: {title}\nSummary: {summary}\n"
        "Constraints: 1-2 punchy lines, clear CTA, 3-5 relevant hashtags."
    )
    return llm.invoke(prompt).content

def _post_via_webhook_impl(text: str, image_path: Optional[str] = None) -> str:
    WEBHOOK_URL = os.environ["SOCIAL_WEBHOOK_URL"]
    files = {"image": open(image_path, "rb")} if image_path else None
    r = requests.post(WEBHOOK_URL, data={"text": text}, files=files, timeout=60)
    r.raise_for_status()
    return "Posted"

# ---------- OPTION 1: Try different CrewAI imports ----------
# try:
#     # Try the main crewai package first
#     from crewai.tools import BaseTool
#     print("Using crewai.tools.BaseTool")
# except ImportError:
#     try:
#         # Try crewai_tools package
#         from crewai_tools import BaseTool
#         print("Using crewai_tools.BaseTool")
#     except ImportError:
#         try:
#             # Try alternative import path
#             from crewai.tool import BaseTool
#             print("Using crewai.tool.BaseTool")
#         except ImportError:
#             # If all imports fail, create a simple base class
#             print("Creating custom BaseTool")
#             class BaseTool:
#                 name: str = "base_tool"
#                 description: str = "Base tool class"

#                 def _run(self, *args, **kwargs):
#                     raise NotImplementedError("Subclasses must implement _run method")


class MakeSocialCopyTool(BaseTool):
    name: str = "make_social_copy"
    description: str = "Create a short, platform-optimized social post from a title and summary. Args: title (str), summary (str), platform (str, optional)"

    def _run(self, title: str, summary: str, platform: str = "linkedin") -> str:
        return _make_social_copy_impl(title, summary, platform)

class PostViaWebhookTool(BaseTool):
    name: str = "post_via_webhook"
    description: str = "Publish a post (and optional image) via a webhook integration. Args: text (str), image_path (str, optional)"

    def _run(self, text: str, image_path: Optional[str] = None) -> str:
        return _post_via_webhook_impl(text, image_path)

# Create tool instances
# generate_image_local = GenerateImageTool()
make_social_copy = MakeSocialCopyTool()
post_via_webhook = PostViaWebhookTool()


# 🔁 Function-Based Tool Fallbacks
- Supplies plain functions equivalent to the class-based tools.
- Some CrewAI versions prefer function tools; this keeps things compatible.


# 👥 Agents: Researcher, Writer, Social Strategist
- **Researcher**: gathers 5 current insights (no web tool here; relies on LLM knowledge unless you add a search tool).
- **Writer**: converts notes into a short article.
- **Social Media Strategist**: uses the tools to create platform-specific posts, optionally publish via webhook.
- Includes a try/except to attach class-based tools first, then fall back to function-based tools.


In [ ]:

# ---------- OPTION 2: Alternative - Use function-based tools ----------
def create_function_tools():
    """Alternative approach using function-based tools that CrewAI can accept"""

    def make_social_copy_func(title: str, summary: str, platform: str = "linkedin") -> str:
        """Create a short, platform-optimized social post from a title and summary."""
        return _make_social_copy_impl(title, summary, platform)

    def post_via_webhook_func(text: str, image_path: Optional[str] = None) -> str:
        """Publish a post (and optional image) via a webhook integration."""
        return _post_via_webhook_impl(text, image_path)

    # return generate_image_local_func, make_social_copy_func, post_via_webhook_func
    return make_social_copy_func, post_via_webhook_func

# Create function-based tools as backup
# generate_image_func, make_social_func, post_webhook_func = create_function_tools()
make_social_func, post_webhook_func = create_function_tools()

# ---------- Agents ----------
from crewai import Agent
from langchain_openai import ChatOpenAI

# Make sure to set your OpenAI API key
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

researcher = Agent(
    role="Researcher",
    goal="Collect 5 up-to-date, credible insights on the topic.",
    backstory="Finds and synthesizes sources.",
    allow_delegation=False,
    llm=llm,
)

writer = Agent(
    role="Writer",
    goal="Draft a 600-900 word article from notes with clear structure.",
    backstory="Senior technical writer.",
    allow_delegation=False,
    llm=llm,
)

# Try using the class-based tools first, fallback to functions if needed
# try:

social_agent = Agent(
    role="Social Media Strategist",
    goal="Adapt article into a platform-optimized post and (optionally) publish.",
    backstory="Knows tone/length/hashtag best practices.",
    tools=[make_social_copy, post_via_webhook],
    allow_delegation=False,
    llm=llm,
)
print("Successfully created agents with class-based tools")

# except Exception as e:
#     print(f"Class-based tools failed: {e}")
#     print("Trying function-based tools...")

#     social_agent = Agent(
#         role="Social Media Strategist",
#         goal="Adapt article into a platform-optimized post and (optionally) publish.",
#         backstory="Knows tone/length/hashtag best practices.",
#         tools=[make_social_func, post_webhook_func],
#         allow_delegation=False,
#         llm=llm,
#     )
#     print("Successfully created agents with function-based tools")

print("All agents created successfully!")

Successfully created agents with class-based tools
All agents created successfully!


# 📝 Tasks & Context Flow
- `t1` (Researcher): produce 5 concise insights with short citations.
- `t2` (Writer): create a ~250–300 word article from `t1`'s output (title, summary, sections, conclusion).
- `t3` (Social): extract title/summary from the article (context=`t2`), produce LinkedIn + Twitter versions, **don’t publish** (just show drafts).

> `context=[t1]` or `[t2]` passes upstream outputs downstream.


In [ ]:
# ---------- Create Tasks ----------
t1 = Task(
    description="Research the topic: 'State of Canada's economic growth in the 21st century'. Find 5 key insights with current developments, challenges, and opportunities. Include brief references to sources.",
    agent=researcher,
    expected_output="Bulleted list of 5 insights with short citations or source references.",
)

t2 = Task(
    description="Write a structured 250 to 300 word article from the research notes. Include: compelling title, introduction, 3-4 main sections, and conclusion. Start with a brief summary paragraph.",
    agent=writer,
    expected_output="Complete markdown article with title, summary paragraph, and well-structured sections.",
    context=[t1],
)


t3 = Task(
    description="""Create social media posts for the article:
    1. Use make_social_copy to create a LinkedIn version (title from article, summary from article, platform='linkedin')
    2. Use make_social_copy to create an X/Twitter version (title from article, summary from article, platform='twitter')
    3. For testing purposes, do NOT publish automatically - just show what the posts would look like

    Extract the title and summary from the article context to use with the tools.""",
    agent=social_agent,
    expected_output="LinkedIn post version, X/Twitter post version, and publishing status.",
    context=[t1, t2],
)

# 🚀 Build Crew & Run
- Creates the `Crew` with your agents and tasks.
- `verbose=True` to see execution traces.
- Calls `crew.kickoff()` to execute the workflow end-to-end.
- Catches and prints any runtime errors (e.g., bad API key, rate limit, missing webhook).


In [ ]:
crew = Crew(
    agents=[researcher, writer, social_agent],
    tasks=[t1, t2, t3],
    verbose=True  # Set to True or 2 for detailed output
)

print("🚀 Starting CrewAI workflow...")
print("=" * 50)

try:
    result = crew.kickoff()
    print("\n" + "=" * 50)
    print("✅ WORKFLOW COMPLETED!")
    print("=" * 50)
    print(result)
except Exception as e:
    print(f"❌ Error running crew: {e}")
    print("Check your API keys and environment setup.")

🚀 Starting CrewAI workflow...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 32229cc6-104c-4fc3-918b-7d676d282299                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Task: Research the topic: 'State of Canada's economic growth in the 21st century'. Find 5 key insights with    │
│  current developments, challenges, and opportunities. Include brief references to sources.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Researcher                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  - **Moderate Economic Growth Rates**: Canada's economy has experienced moderate growth since the early 2000s,  │
│  averaging about 2% annually. The COVID-19 pandemic led to a sharp contraction in 2020, but recovery has been   │
│  underway, with growth projected at approximately 3.4% for 2022 and about 2% for 2023 as per the International  │
│  Monetary Fund (IMF) forecasts (IMF, 2022).                                                                     │
│                                                                                                                 │
│  - **Resource Sector Dominance**: Canada's economy has shown resilience, particularly in the resource sector,   │
│  which includes oil, gas, and mining. With commodity prices fluctuating, the shift to renewable energy sources  │
│  presents both challenges and opportunities for Canadian resource firms to pivot towards sustainable practices  │
│  (Natural Resources Canada, 2023).                                                                              │
│                                                                                                                 │
│  - **Labor Market Tightness**: Canada faces a tight labor market, which poses a challenge for businesses due    │
│  to severe labor shortages in various sectors. This reflects an aging population and declining birth rates.     │
│  Efforts are being made to address this through immigration policies aimed at filling labor gaps (Statistics    │
│  Canada, 2023).                                                                                                 │
│                                                                                                                 │
│  - **Technological Innovation and Transition**: The 21st century has seen a surge in technological innovation,  │
│  especially in the digital sector. Initiatives to foster a tech-friendly environment and support startups       │
│  through funding and resources are underway. The tech sector is expected to significantly contribute to job     │
│  creation and economic growth going forward (Canadian Digital Ecosystem, 2023).                                 │
│                                                                                                                 │
│  - **Trade Relations and NAFTA 2.0**: Following the renegotiation of NAFTA into the United                      │
│  States-Mexico-Canada Agreement (USMCA), Canada's trade relations have adapted and shifted. This has opened     │
│  new markets for Canadian goods while also requiring a rapid response to global trade shifts, especially in     │
│  light of rising protectionist policies (Government of Canada, 2023).                                           │
│                                                                                                                 │
│  References:                                                                                                    │
│  - International Monetary Fund (IMF). (2022). World Economic Outlook.                                           │
│  - Natural Resources Canada. (2023). Canada's Natural Resource Sectors.                                         │
│  - Statistics Canada. (2023). Job Vacancy and Wage Survey.                                                      │
│  - Canadian Digital Ecosystem. (2023). Canada’s Competi

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Social Media Strategist                                                                                 │
│                                                                                                                 │
│  Thought: Thought: Now I will create the X/Twitter version of the post.                                         │
│                                                                                                                 │
│  Using Tool: make_social_copy                                                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Social Media Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  LinkedIn Post Version:                                                                                         │
│  🔍 Discover how Canada’s economy is thriving amidst global challenges! Explore the driving forces behind its   │
│  resilience and untapped opportunities in my latest article.                                                    │
│                                                                                                                 │
│  👉 Read more here: [link]                                                                                      │
│                                                                                                                 │
│  #CanadaEconomy #EconomicGrowth #Innovation #Trade #Opportunities                                               │
│                                                                                                                 │
│  X/Twitter Post Version:                                                                                        │
│  Explore how Canada’s economy thrives against global challenges! Discover the key drivers of resilience and     │
│  opportunity in our latest article. 🌍📈 Read now! 👉 [link] #CanadaEconomy #Resilience #Innovation #Trade      │
│  #EconomicGrowth                                                                                                │
│                                                                                                                 │
│  Publishing Status: Not Published (for testing purposes only)                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()


✅ WORKFLOW COMPLETED!
LinkedIn Post Version:
🔍 Discover how Canada’s economy is thriving amidst global challenges! Explore the driving forces behind its resilience and untapped opportunities in my latest article. 

👉 Read more here: [link] 

#CanadaEconomy #EconomicGrowth #Innovation #Trade #Opportunities

X/Twitter Post Version:
Explore how Canada’s economy thrives against global challenges! Discover the key drivers of resilience and opportunity in our latest article. 🌍📈 Read now! 👉 [link] #CanadaEconomy #Resilience #Innovation #Trade #EconomicGrowth

Publishing Status: Not Published (for testing purposes only)


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 73f789ca-3c3c-4fe1-b87c-3721ef1c8f8d                                                                     │
│  Agent: Social Media Strategist                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯